<a href="https://colab.research.google.com/github/erric-syd/installment-adjustment/blob/main/Repayment_Adjustment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Background:
- Terdapat diff pada loan:loan_amount dan repayment:principal_paid_total
 
Rekomendasi:
- Apabila ada perbedaan, munculkan alert.
- Apabila tidak ada, biarkan saja schedule nya.

Solusi:
- Apabila ada loan yang amount dan repayment ny terdapat "diff" dan repayment melebihi maturity, maka repayment tsb di 0-kan dan di group ke repayment yang sesuai dgn tgl maturity

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# Sample loan: Loan:loan_amount = 1000, tenor = 6
dt_range = pd.date_range(start='2019-12', end='2020-06', freq='M') + pd.Timedelta(days=15)
dt = []
for x in range(len(dt_range)):
  dt.append(pd.to_datetime(dt_range.date[x].strftime("%Y-%m-%d")))

df_repayment = pd.DataFrame(dt, columns=['repayment_date'])

In [3]:
df_repayment['loan_id'] = '1'
df_repayment['status_repayment'] = 'REPAYMENT_COMPLETED'
df_repayment['principal_paid'] = int(np.round(1000/6))
df_repayment['principal_paid_total'] = df_repayment.groupby(by=['loan_id'])['principal_paid'].cumsum().values.astype(int)
df_repayment['principal_total'] = df_repayment['principal_paid'].sum()
df_repayment['principal_outstanding'] = df_repayment['principal_total'] - df_repayment['principal_paid_total']
df_repayment['loan_amount'] = 1000
df_repayment['sold_date'] = pd.to_datetime('2020-01-01')
df_repayment['maturity_date'] = pd.to_datetime('2020-05-15')
df_repayment['principal_paid_origin'] = df_repayment['principal_paid']

In [4]:
# Custom val
# df_repayment['principal_paid'] = ['167', '167', '167', '167', '167', '200']

In [5]:
df_repayment

,repayment_date,loan_id,status_repayment,principal_paid,principal_paid_total,principal_total,principal_outstanding,loan_amount,sold_date,maturity_date,principal_paid_origin
0,2020-01-15,1,REPAYMENT_COMPLETED,167,167,1002,835,1000,2020-01-01,2020-05-15,167
1,2020-02-15,1,REPAYMENT_COMPLETED,167,334,1002,668,1000,2020-01-01,2020-05-15,167
2,2020-03-15,1,REPAYMENT_COMPLETED,167,501,1002,501,1000,2020-01-01,2020-05-15,167
3,2020-04-15,1,REPAYMENT_COMPLETED,167,668,1002,334,1000,2020-01-01,2020-05-15,167
4,2020-05-15,1,REPAYMENT_COMPLETED,167,835,1002,167,1000,2020-01-01,2020-05-15,167
5,2020-06-15,1,REPAYMENT_COMPLETED,167,1002,1002,0,1000,2020-01-01,2020-05-15,167


# df_dateminmax

Min & Max repayment date

In [6]:
# Get repayment_date min & max from df_repayment
# Rename multi-index column name
col = ['loan_id', 'repayment_date', 'principal_total', 'loan_amount']
df_dateminmax = df_repayment.loc[:, col].groupby(by=['loan_id'], as_index=False).agg({'repayment_date':[np.max, np.min]})
df_dateminmax.columns = df_dateminmax.columns.to_flat_index()
df_dateminmax.rename(columns={('loan_id', ''): 'loan_id', ('repayment_date', 'amax'): 'max_repayment_date', ('repayment_date', 'amin'): 'min_repayment_date'}, inplace=True)

In [7]:
# Merge df_repayment & df_dateminmax, new column max_repayment_date & min_repayment_date
df_repayment = df_repayment.merge(df_dateminmax, how='left', left_on='loan_id', right_on='loan_id')

In [8]:
##
col = ['loan_id', 'repayment_date', 'principal_paid_origin', 'principal_paid', 'principal_paid_total', 'principal_outstanding', 'principal_total', 'loan_amount', 'sold_date', 'maturity_date',  'max_repayment_date', 'min_repayment_date']
df_repayment.loc[:, col].head(2)

,loan_id,repayment_date,principal_paid_origin,principal_paid,principal_paid_total,principal_outstanding,principal_total,loan_amount,sold_date,maturity_date,max_repayment_date,min_repayment_date
0,1,2020-01-15,167,167,167,835,1002,1000,2020-01-01,2020-05-15,2020-06-15,2020-01-15
1,1,2020-02-15,167,167,334,668,1002,1000,2020-01-01,2020-05-15,2020-06-15,2020-01-15


# PREVIOUS REPAYMENT DATE

In [9]:
# Create new column previous_repayment_date
# Filter if repayment_date is equal to min_repayment_date
# True: loan_sold_date
# False: use previous repayment_date
cond_x = df_repayment['repayment_date'] == df_repayment['min_repayment_date']
choice_x = df_repayment['sold_date'].values
prev_date = df_repayment['repayment_date'].shift(1)

df_repayment['previous_repayment_date'] = np.where(cond_x, choice_x, prev_date)

In [10]:
##
col = ['loan_id', 'repayment_date', 'previous_repayment_date', 'maturity_date']
df_repayment.loc[:, col].head(2)

,loan_id,repayment_date,previous_repayment_date,maturity_date
0,1,2020-01-15,2020-01-01,2020-05-15
1,1,2020-02-15,2020-01-15,2020-05-15


# REPAYMENT LAST CYCLE

In [11]:
# Check whether repayment already reach its last cycle date by using loan_maturity_date as benchmark
# True: repayment inside cycle date
# False: repayment outside cycle date
range_1 = df_repayment['previous_repayment_date'] + pd.Timedelta(days=1)
range_2 = df_repayment['repayment_date']
df_repayment['repayment_last_cycle'] = df_repayment['maturity_date'].between(range_1, range_2)

In [12]:
##
col = ['loan_id', 'repayment_date', 'previous_repayment_date', 'maturity_date', 'repayment_last_cycle']
df_repayment.loc[:, col]

,loan_id,repayment_date,previous_repayment_date,maturity_date,repayment_last_cycle
0,1,2020-01-15,2020-01-01,2020-05-15,False
1,1,2020-02-15,2020-01-15,2020-05-15,False
2,1,2020-03-15,2020-02-15,2020-05-15,False
3,1,2020-04-15,2020-03-15,2020-05-15,False
4,1,2020-05-15,2020-04-15,2020-05-15,True
5,1,2020-06-15,2020-05-15,2020-05-15,False


# CONDITIONS

In [13]:
##
col = ['loan_id', 'principal_total', 'loan_amount', 'maturity_date', 'repayment_date', 'max_repayment_date', 'repayment_last_cycle']
df_repayment.loc[:, col].tail(2)

,loan_id,principal_total,loan_amount,maturity_date,repayment_date,max_repayment_date,repayment_last_cycle
4,1,1002,1000,2020-05-15,2020-05-15,2020-06-15,True
5,1,1002,1000,2020-05-15,2020-06-15,2020-06-15,False


In [14]:
# nb: Maturity_date loan itu harus sama dengan last_repayment_date

# Check if repayment principal_total not equal as loan_amount
cond_1 = df_repayment['principal_total'] != df_repayment['loan_amount']

# Check if loan_maturity_date is less than or equal to repayment_date, loan already beyond maturity
cond_2 = df_repayment['maturity_date'].values <= df_repayment['repayment_date'].values

# Check if loan_maturity_date is greater than repayment_date, loan yet matured.
cond_3 = df_repayment['repayment_date'].values < df_repayment['maturity_date'].values

# Check if repayment_date equal to max_repayment_date (guard a.k.a kondisi yg ngejagain supaya row yang ditampilkan hanya repayment terakhir)
cond_4 = df_repayment['repayment_date'] == df_repayment['max_repayment_date']

# Repayment inside last cycle period 
cond_cycle_t = (df_repayment['repayment_last_cycle'] == True)

# Repayment outside last cycle period
cond_cycle_f = (df_repayment['repayment_last_cycle'] == False)

# cond_premature = Recalculate repayment that have diff (cond_1), loan yet to mature (cond_3), only the last repayment (cond_4) a.k.a schedule matured before maturity_date, and outside the repayment cycle (cond_cycle_f)
# cond_mature = Recalculate schedule that have diff amount (cond_1), maturity_date less or equal to repayment_date (cond_2) and inside the repayment cycle (cond_cycle_t)
# cond_already_matured = For schedule that already beyond its maturity (cond_2), and outside the repayment cycle (cond_cycle_f)
cond_premature = cond_1 & cond_3 & cond_4 & cond_cycle_f
cond_mature = cond_1 & cond_2 & cond_cycle_t
cond_already_matured = cond_1 & cond_2 & cond_cycle_f

# Check principal

In [15]:
# Create new column principal_check
# cond_1 = check if loan_amount not equal to principal_total a.k.a diff
# cond_2 = flag when loan_maturity_date is less than or equal to repayment_date
# cond_4 = Get the max repayment_date, put "need to check" if there is a diff
df_repayment['principal_check'] = np.where(
    cond_1 & (cond_2 | cond_4), 
    'NEED TO CHECK', '')

In [16]:
##
col = ['loan_id', 'principal_total', 'loan_amount', 'maturity_date', 'repayment_date', 'max_repayment_date', 'repayment_last_cycle', 'principal_check']
df_repayment.loc[:, col]

,loan_id,principal_total,loan_amount,maturity_date,repayment_date,max_repayment_date,repayment_last_cycle,principal_check
0,1,1002,1000,2020-05-15,2020-01-15,2020-06-15,False,
1,1,1002,1000,2020-05-15,2020-02-15,2020-06-15,False,
2,1,1002,1000,2020-05-15,2020-03-15,2020-06-15,False,
3,1,1002,1000,2020-05-15,2020-04-15,2020-06-15,False,
4,1,1002,1000,2020-05-15,2020-05-15,2020-06-15,True,NEED TO CHECK
5,1,1002,1000,2020-05-15,2020-06-15,2020-06-15,False,NEED TO CHECK


# Count Anomaly Repayment

In [17]:
# Filter principal_check column that contain 'NEED TO CHECK' string
row = df_repayment['principal_check'].str.contains('NEED TO CHECK')

In [18]:
# Count anomaly repayment schedule per loan_id
col = ['loan_id', 'principal_check']
df_count_anomaly = df_repayment.loc[row, col].groupby(by=['loan_id'])['principal_check'].count().to_frame(name='count_anomaly').reset_index()

In [19]:
df_count_anomaly

,loan_id,count_anomaly
0,1,2


In [20]:
# Get repayment_date that flagged as "Anomaly"
col = ['loan_id', 'repayment_date']
df_count_anomaly = df_count_anomaly.merge(df_repayment.loc[row, col], how='left', left_on='loan_id', right_on='loan_id')

In [21]:
df_count_anomaly

,loan_id,count_anomaly,repayment_date
0,1,2,2020-05-15
1,1,2,2020-06-15


In [22]:
# Merge df_repayment & df_count_anomaly
df_repayment = df_repayment.merge(df_count_anomaly, how='left', left_on=['loan_id', 'repayment_date'], right_on=['loan_id', 'repayment_date'])

In [23]:
# Create new column count_anomaly & fill the new column null values with 0
df_repayment['count_anomaly'] = df_repayment['count_anomaly'].fillna(value = 0)

In [24]:
##
col = ['loan_id', 'principal_check', 'count_anomaly']
df_repayment.loc[:, col]

,loan_id,principal_check,count_anomaly
0,1,,0.000
1,1,,0.000
2,1,,0.000
3,1,,0.000
4,1,NEED TO CHECK,2.000
5,1,NEED TO CHECK,2.000


# PRINCIPAL_PAID_recalc



In [25]:
df_repayment.dtypes

repayment_date             datetime64[ns]
loan_id                            object
status_repayment                   object
principal_paid                      int64
principal_paid_total                int64
principal_total                     int64
principal_outstanding               int64
loan_amount                         int64
sold_date                  datetime64[ns]
maturity_date              datetime64[ns]
principal_paid_origin               int64
max_repayment_date         datetime64[ns]
min_repayment_date         datetime64[ns]
previous_repayment_date    datetime64[ns]
repayment_last_cycle                 bool
principal_check                    object
count_anomaly                     float64
dtype: object

In [26]:
# Diff amount between loan_amount & repayment principal_total
diff = df_repayment['loan_amount'] - df_repayment['principal_total']

In [37]:
# Create new column principal_paid_recalc
# Condition[1]: Recalculate schedule that have diff amount (cond_1), maturity_date less or equal to repayment_date (cond_2), and inside the repayment cycle (cond_cycle_t);
#               Or Recalculate repayment that doesn't pass through the maturity_date (cond_3) & only the last repayment (cond_4) a.k.a schedule matured before maturity_date.
# choice_1: Principal_paid times count_anomaly because principal_total was sum of all principal_paid including the anomaly schedule. 
# After updated, you need count_anomaly so that principal_paid_recalc sum equal to the principal_total 
# add that with the diff from (loan_amount - principal_total) and you get the sum of principal_paid_recalc equal to loan_amount
# Condition[2]: For schedule that already beyond its maturity (cond_2), and outside the repayment cycle (cond_cycle_f)
# choice_2: set to zero because schedule already beyond maturity_date
# Else
# choice_default: equal to principal_paid column
choice_1 = (df_repayment['principal_paid'] * df_repayment['count_anomaly']) + diff
choice_default = df_repayment['principal_paid']
conditions = [
              cond_mature | cond_premature,
              cond_already_matured
]
choices = [
           choice_1,
           0.000
]
df_repayment['principal_paid_recalc'] = np.select(conditions, choices, default = choice_default)
df_repayment['principal_paid_recalc'] = df_repayment['principal_paid_recalc'].astype(int)

In [38]:
##
col = ['loan_id', 'maturity_date', 'repayment_date', 'max_repayment_date', 'principal_paid', 'principal_paid_recalc', 'principal_total', 'loan_amount', 'count_anomaly']
df_repayment.loc[:, col]

,loan_id,maturity_date,repayment_date,max_repayment_date,principal_paid,principal_paid_recalc,principal_total,loan_amount,count_anomaly
0,1,2020-05-15,2020-01-15,2020-06-15,167,167,1002,1000,0.000
1,1,2020-05-15,2020-02-15,2020-06-15,167,167,1002,1000,0.000
2,1,2020-05-15,2020-03-15,2020-06-15,167,167,1002,1000,0.000
3,1,2020-05-15,2020-04-15,2020-06-15,167,167,1002,1000,0.000
4,1,2020-05-15,2020-05-15,2020-06-15,167,332,1002,1000,2.000
5,1,2020-05-15,2020-06-15,2020-06-15,167,0,1002,1000,2.000


In [39]:
##
col = ['loan_id', 'maturity_date', 'repayment_date', 'principal_paid', 'principal_total', 'loan_amount', 'max_repayment_date', 'principal_paid_recalc']
df_repayment.loc[:, col].groupby(by=['loan_id'])['principal_paid_recalc'].sum()

loan_id
1    1000
Name: principal_paid_recalc, dtype: int64

In [40]:
##
col = ['loan_id', 'maturity_date', 'repayment_date', 'principal_paid', 'principal_total', 'loan_amount', 'max_repayment_date', 'principal_paid_recalc']
df_repayment.loc[:, col].groupby(by=['loan_id'])['principal_paid'].sum()

loan_id
1    1002
Name: principal_paid, dtype: int64

# PRINCIPAL_PAID_TOTAL_recalc

In [41]:
 # Create new column of cumulative sum principal_paid_total_recalc
 df_repayment['principal_paid_total_recalc'] = df_repayment.groupby(by=['loan_id'])['principal_paid_recalc'].cumsum().values

In [42]:
##
col = ['loan_id', 'principal_paid_recalc', 'principal_paid_total_recalc']
df_repayment.loc[:, col]

,loan_id,principal_paid_recalc,principal_paid_total_recalc
0,1,167,167
1,1,167,334
2,1,167,501
3,1,167,668
4,1,332,1000
5,1,0,1000


# PRINCIPAL_TOTAL_recalc

In [43]:
# Check if repayment principal_total not equal to loan_amount
# Sum principal_paid_recalc then, rename to principal_total_recalc
row = (df_repayment['principal_total'] != df_repayment['loan_amount'])
df_principal_total_recalc = df_repayment.loc[row, :].groupby(by=['loan_id'])['principal_paid_recalc'].sum().to_frame(name='principal_total_recalc').reset_index()

In [49]:
df_principal_total_recalc

,loan_id,principal_total_recalc
0,1,1000


In [44]:
# Merge df_repayment & df_principal_total_recalc, new column principal_paid_recalc
df_repayment = df_repayment.merge(df_principal_total_recalc, how='left', left_on='loan_id', right_on='loan_id')

In [45]:
# Fill the principal_total_recalc column null values by using principal_total
# Untuk diisi dengan principal_total yang loan nya tidak di adjust alias loan ny bener.
df_repayment['principal_total_recalc'] = df_repayment['principal_total_recalc'].fillna(value = df_repayment['principal_total'])

In [48]:
##
col = ['loan_id', 'loan_amount', 'repayment_date', 'maturity_date', 'principal_check', 'principal_paid', 'principal_paid_recalc', 'principal_total', 'principal_total_recalc']
df_repayment.loc[:, col]

,loan_id,loan_amount,repayment_date,maturity_date,principal_check,principal_paid,principal_paid_recalc,principal_total,principal_total_recalc
0,1,1000,2020-01-15,2020-05-15,,167,167,1002,1000
1,1,1000,2020-02-15,2020-05-15,,167,167,1002,1000
2,1,1000,2020-03-15,2020-05-15,,167,167,1002,1000
3,1,1000,2020-04-15,2020-05-15,,167,167,1002,1000
4,1,1000,2020-05-15,2020-05-15,NEED TO CHECK,167,332,1002,1000
5,1,1000,2020-06-15,2020-05-15,NEED TO CHECK,167,0,1002,1000


# PRINCIPAL_OUTSTANDING_recalc

In [50]:
# Create new column principal_outstanding_recalc
df_repayment['principal_outstanding_recalc'] = df_repayment['principal_total_recalc'] - df_repayment['principal_paid_total_recalc']

In [51]:
##
col = ['loan_id', 'principal_outstanding_recalc', 'principal_paid_recalc', 'principal_paid_total_recalc', 'principal_total_recalc']
df_repayment.loc[:, col]

,loan_id,principal_outstanding_recalc,principal_paid_recalc,principal_paid_total_recalc,principal_total_recalc
0,1,833,167,167,1000
1,1,666,167,334,1000
2,1,499,167,501,1000
3,1,332,167,668,1000
4,1,0,332,1000,1000
5,1,0,0,1000,1000


In [52]:
## principal_paid_recalc
col = ['loan_id', 'principal_paid_recalc']
df_repayment.loc[:, col].groupby(by=['loan_id'])['principal_paid_recalc'].sum()

loan_id
1    1000
Name: principal_paid_recalc, dtype: int64

# Check Interest

In [54]:
# Create new column interest_check
# Condition[1]: Schedule already matured, ex: maturity_date jan_1, last repayment_date feb 15
# choice_1: ALREADY MATURED
# Condition[2]: Schedule is in range of maturity_date or inside cycle date, ex: maturity_date jan 1, last repayment_date jan 15.
# choice_2: MATURE
# Condition[3]: Schedule mature before maturity_date, ex: maturity_date feb 1, last repayment_date jan 15
# choice_3: PREMATURE; means that there is none of schedule that goes beyond maturity_date a.k.a gk ada repayment yg kelebihan jadwal, jadi langsung cek repayment terakhir aja
conditions = [
              cond_already_matured,
              cond_mature,
              cond_premature
]
choices = [
           'ALREADY MATURED',
           'MATURE',
           'PREMATURE'
]
df_repayment['interest_check'] = np.select(conditions, choices, default = '')

In [57]:
##
col = ['loan_id', 'loan_amount', 'repayment_date', 'maturity_date', 'max_repayment_date', 'min_repayment_date', 'interest_check', 'principal_check']
df_repayment.loc[:, col]

,loan_id,loan_amount,repayment_date,maturity_date,max_repayment_date,min_repayment_date,interest_check,principal_check
0,1,1000,2020-01-15,2020-05-15,2020-06-15,2020-01-15,,
1,1,1000,2020-02-15,2020-05-15,2020-06-15,2020-01-15,,
2,1,1000,2020-03-15,2020-05-15,2020-06-15,2020-01-15,,
3,1,1000,2020-04-15,2020-05-15,2020-06-15,2020-01-15,,
4,1,1000,2020-05-15,2020-05-15,2020-06-15,2020-01-15,MATURE,NEED TO CHECK
5,1,1000,2020-06-15,2020-05-15,2020-06-15,2020-01-15,ALREADY MATURED,NEED TO CHECK


# Result

In [60]:
# Display necessary column for prod
col = ['loan_id', 'repayment_date', 
       'principal_paid_origin', 'principal_paid_recalc', 'principal_paid_total_recalc', 'principal_outstanding_recalc', 'principal_total_recalc', 
       'status_repayment', 'interest_check', 'principal_check']
df_repayment.loc[:, col]

,loan_id,repayment_date,principal_paid_origin,principal_paid_recalc,principal_paid_total_recalc,principal_outstanding_recalc,principal_total_recalc,status_repayment,interest_check,principal_check
0,1,2020-01-15,167,167,167,833,1000,REPAYMENT_COMPLETED,,
1,1,2020-02-15,167,167,334,666,1000,REPAYMENT_COMPLETED,,
2,1,2020-03-15,167,167,501,499,1000,REPAYMENT_COMPLETED,,
3,1,2020-04-15,167,167,668,332,1000,REPAYMENT_COMPLETED,,
4,1,2020-05-15,167,332,1000,0,1000,REPAYMENT_COMPLETED,MATURE,NEED TO CHECK
5,1,2020-06-15,167,0,1000,0,1000,REPAYMENT_COMPLETED,ALREADY MATURED,NEED TO CHECK


# ggwp mekar team :)